In [1]:
import numpy as np
import os
import sys
import tensorflow as tf

2025-03-29 08:39:09.627861: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-29 08:39:09.631697: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-29 08:39:09.643983: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743233949.665281   93660 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743233949.670537   93660 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-29 08:39:09.693409: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [ ]:
# Work-around for loading a module from a parent folder in Jupyter/Notebooks
parent_dir = os.path.abspath(os.path.join('..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from modules.utils import get_convolution_layer_expected_output_info
from modules.autoencoders_keras import VariationalAutoencoder

In [3]:
n_filters = 10
#padding = 'valid'  # No padding to the input
padding = 'same'    # Apply padding to the input

k=40
s=30

x_1d = np.random.rand(15, 120, 3)
k_1d = (k,)
s_1d = (s,)

x_2d = np.random.rand(15, 120, 120, 3)
k_2d = (k,k)
s_2d = (s,s)

x_3d = np.random.rand(15, 120, 120, 120, 3)
k_3d = (k,k,k)
s_3d = (s,s,s)

In [4]:
from keras import layers

def get_convolution_layer_output_info(input, kernel_shape, stride_shape, n_filters=10, padding='valid'):
    n_dims = len(input.shape[1:-1])
    if n_dims == 1:
        conv = layers.Conv1D(
            filters=n_filters,
            kernel_size=kernel_shape,
            strides=stride_shape,
            padding=padding,
            name=f"conv_{n_dims}d_layer"
        )
    elif n_dims == 2:
        conv = layers.Conv2D(
            filters=n_filters,
            kernel_size=kernel_shape,
            strides=stride_shape,
            padding=padding,
            name=f"conv_{n_dims}d_layer"
        )
    elif n_dims == 3:
        conv = layers.Conv3D(
            filters=n_filters,
            kernel_size=kernel_shape,
            strides=stride_shape,
            padding=padding,
            name=f"conv_{n_dims}d_layer"
        )
    else:
        return (None,),(None,)
    y = conv(input)
    output_shape = y.shape[1:]

    #def get_padding_from_output(input, output, stride, kernel):
    #    p = (((output-1)*stride)+kernel-input)/2
    #    return int(p)
    #padding_shape = tuple([get_padding_from_output(input.shape[1:-1][i], ow, stride_shape[i], kernel_shape[i]) for i, ow in enumerate(output_shape[:-1])])

    #padding_shape = tuple([abs(int((((ow-1)*stride_shape[i])+kernel_shape[i]-input.shape[1:-1][i])/2)) for i, ow in enumerate(output_shape[:-1])])

    #return output_shape, padding_shape, y
    return output_shape, y

# Convolution 1D

In [5]:
x = x_1d
kernel_shape = k_1d
stride_shape = s_1d

name = 'Conv 1D'

print(f'Input shape ({name}): {x.shape[1:]}')

# Understaning the output shape of a Conv layer
expected_output_shape, expected_padding_shape = get_convolution_layer_expected_output_info(x, kernel_shape, stride_shape, n_filters, padding)
print(f'Expected output shape ({name}): {expected_output_shape}, padding: {expected_padding_shape}')

output_shape, y = get_convolution_layer_output_info(x, kernel_shape, stride_shape, n_filters, padding)
print(f'Output shape ({name}): {output_shape}')

Input shape (Conv 1D): (120, 3)
Expected output shape (Conv 1D): (4, 10), padding: (10,)
Output shape (Conv 1D): (4, 10)


2025-03-29 08:39:13.592354: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


# Convolution 2D

In [6]:
x = x_2d
kernel_shape = k_2d
stride_shape = s_2d

name = 'Conv 2D'

print(f'Input shape ({name}): {x.shape[1:]}')

# Understaning the output shape of a Conv layer
expected_output_shape, expected_padding_shape = get_convolution_layer_expected_output_info(x, kernel_shape, stride_shape, n_filters, padding)
print(f'Expected output shape ({name}): {expected_output_shape}, padding: {expected_padding_shape}')

output_shape, y = get_convolution_layer_output_info(x, kernel_shape, stride_shape, n_filters, padding)
print(f'Output shape ({name}): {output_shape}')

Input shape (Conv 2D): (120, 120, 3)
Expected output shape (Conv 2D): (4, 4, 10), padding: (10, 10)
Output shape (Conv 2D): (4, 4, 10)


# Convolution 3D

In [7]:
x = x_3d
kernel_shape = k_3d
stride_shape = s_3d

name = 'Conv 3D'

print(f'Input shape ({name}): {x.shape[1:]}')

# Understaning the output shape of a Conv layer
expected_output_shape, expected_padding_shape = get_convolution_layer_expected_output_info(x, kernel_shape, stride_shape, n_filters, padding)
print(f'Expected output shape ({name}): {expected_output_shape}, padding: {expected_padding_shape}')

output_shape, y = get_convolution_layer_output_info(x, kernel_shape, stride_shape, n_filters, padding)
print(f'Output shape ({name}): {output_shape}')

Input shape (Conv 3D): (120, 120, 120, 3)
Expected output shape (Conv 3D): (4, 4, 4, 10), padding: (10, 10, 10)
Output shape (Conv 3D): (4, 4, 4, 10)


# Tests

In [8]:
out_shape, pad_shape = get_convolution_layer_expected_output_info(np.random.rand(1, 28, 28, 32), (3,3), (2,2), n_filters=64, padding='same')
display(out_shape, pad_shape)

(14, 14, 64)

(1, 1)

In [9]:
out_shape, pad_shape = get_convolution_layer_expected_output_info(np.random.rand(1, 14, 14, 64), (3,3), (2,2), n_filters=64, padding='same')
display(out_shape, pad_shape)

(7, 7, 64)

(1, 1)

# Analysis

In [15]:
def data_generator(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]

In [16]:
norm_x_train = np.random.normal(3, 2.5, size=(1000, 28, 28, 1))

vae_input_shape = (norm_x_train.shape[1:][0], norm_x_train.shape[1:][1], 1)
vae_filters=(32, 64, 64, 64)
vae_activations=('relu', 'relu', 'relu', 'relu')
vae_kernels=(3, 3, 3, 3)
vae_strides=(1, 2, 2, 1)
vae_latent_space_dim=2

In [17]:
vae_model = VariationalAutoencoder(
    input_shape=vae_input_shape, 
    filters=vae_filters, 
    kernels=vae_kernels, 
    strides=vae_strides, 
    activations=vae_activations,
    latent_space_dim=vae_latent_space_dim,
    model_name="test_vae",
    reconstruction_loss_weight=1000
)
vae_model.summary()

Model: "test_vae"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder (Functional)            │ ((None, 2), (None, 2), │       105,220 │
│                                 │ (None, 2))             │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Functional)            │ (None, 28, 28, 1)      │       120,769 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,989 (882.77 KB)

 Trainable params: 225,989 (882.77 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
epochs = 20
b_size = 128
optimizer = tf.keras.optimizers.get("Adam")
optimizer.learning_rate.assign(.001)

def custom_loss(y_true,y_pred,mean,log_var):
  loss_rec = tf.reduce_mean(tf.reduce_sum(tf.keras.losses.binary_crossentropy(y_true,y_pred), axis = (1,2)))
  loss_reg = -0.5 * (1 + log_var - tf.square(mean) - tf.exp(log_var))  
  return loss_rec+tf.reduce_mean(tf.reduce_sum(loss_reg, axis=1))
  
@tf.function
def training_block(x_batch, encoder_model, decoder_model, vae_model):
  with tf.GradientTape() as recorder:
    z,mean,log_var = encoder_model(x_batch)
    y_pred = decoder_model(z)
    y_true = x_batch
    loss = custom_loss(y_true,y_pred, mean, log_var)  
  partial_derivatives = recorder.gradient(loss, vae_model.trainable_weights)
  optimizer.apply_gradients(zip(partial_derivatives, vae_model.trainable_weights))
  return loss
  
def neuralearn(epochs, train_dataset, encoder_model, decoder_model, vae_model):
  for epoch in range(1,epochs+1):
    print('Training starts for epoch number {}'.format(epoch))
    for step, x_batch in enumerate(train_dataset):
      loss = training_block(x_batch, encoder_model, decoder_model, vae_model)
    print('Training Loss is: ', loss.numpy())
  print('Training Complete!!!')

neuralearn(epochs, data_generator(norm_x_train, batch_size=b_size), vae_model.get_encoder(), vae_model.get_decoder(), vae_model)

Training starts for epoch number 1
Training Loss is:  -10172.646
Training starts for epoch number 2
Training Loss is:  -10172.646
Training starts for epoch number 3
Training Loss is:  -10172.646
Training starts for epoch number 4
Training Loss is:  -10172.646
Training starts for epoch number 5
Training Loss is:  -10172.646
Training starts for epoch number 6
Training Loss is:  -10172.646
Training starts for epoch number 7
Training Loss is:  -10172.646
Training starts for epoch number 8
Training Loss is:  -10172.646
Training starts for epoch number 9
Training Loss is:  -10172.646
Training starts for epoch number 10
Training Loss is:  -10172.646
Training starts for epoch number 11
Training Loss is:  -10172.646
Training starts for epoch number 12
Training Loss is:  -10172.646
Training starts for epoch number 13
Training Loss is:  -10172.646
Training starts for epoch number 14
Training Loss is:  -10172.646
Training starts for epoch number 15
Training Loss is:  -10172.646
Training starts for